In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

from utilidades.validar import validar
from utilidades.graficar import ver_dispersion

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

dfinicial = pd.read_csv("data/train_limpio.csv")
dftest = pd.read_csv("data/test_limpio.csv")

#dfinicial2 = pd.read_csv("data/esteban/train_limpio_ciudades.csv")
#dftest2 = pd.read_csv("data/esteban/test_limpio_ciudades.csv")

xg = pd.read_csv('submits/submit_xg_prueba.csv')

In [3]:
dfinicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 71 columns):
tipodepropiedad_Apartamento                      240000 non-null int64
tipodepropiedad_Casa en condominio               240000 non-null int64
tipodepropiedad_Casa                             240000 non-null int64
tipodepropiedad_Terreno                          240000 non-null int64
tipodepropiedad_Terreno comercial                240000 non-null int64
tipodepropiedad_Local Comercial                  240000 non-null int64
tipodepropiedad_Quinta Vacacional                240000 non-null int64
tipodepropiedad_Oficina comercial                240000 non-null int64
tipodepropiedad_Edificio                         240000 non-null int64
tipodepropiedad_Casa uso de suelo                240000 non-null int64
tipodepropiedad_Local en centro comercial        240000 non-null int64
tipodepropiedad_Bodega comercial                 240000 non-null int64
tipodepropiedad_Otros                

In [4]:
xg.head()

,id,target
0,4941,4831603.00
1,51775,812344.94
2,115253,2203735.00
3,299321,1403080.20
4,173570,596104.40


In [5]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

In [24]:
def generar_subida(DFTRAIN, metodo, DFTEST, nomArchivo, pasarAdisco):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)    
    
    X, y = DFTRAIN.drop( columns = ['precio'] ), DFTRAIN["precio"]

    reg = metodo(
        alpha=0.9, #no cambia nada
        max_depth=4,
        #n_estimators=5, con 5 va mal
        learning_rate=0.5,
        min_samples_leaf=250,
        subsample=0.5,
        validation_fraction=0.1
    )
    reg.fit(X,y)
    
    subida = pd.DataFrame( reg.predict(DFTEST) , columns=["target"])
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    # Heurística: si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    
    # Heurística 2: si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    #Este modelo no necesita las euristicas,pero igual no da muy bien
    
    
    if pasarAdisco:
        subida.to_csv(f"submits/" + nomArchivo, index = False) 
    
    return X, y, subida

In [25]:
dfinicial1 = dfinicial[dfinicial['metrostotales']<225].drop( columns = ['dia'] )
dftest1 = dftest.drop( columns = ['dia'] )

#dfinicial1 = dfinicial2.drop( columns = [ 'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos',  'escuelascercanas'
#                           ])                                        
 
#dftest1 = dftest2.drop( columns = [  'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos', 'escuelascercanas'
#                           ])   

X, y, subida = generar_subida( dfinicial1, GradientBoostingRegressor, dftest1, "prueba_andres2.csv", True)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(xg, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )

390554.0

In [8]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor, False)

Error de fold N°0 es : 648460.5723698041
Error de fold N°1 es : 651863.0967218477
Error de fold N°2 es : 652079.2404494584
Error de fold N°3 es : 652944.970950236
Error de fold N°4 es : 650479.0082130575


In [17]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor, alpha=0.9,max_depth=4)

Error de fold N°0 es : 607840.9459540583
Error de fold N°1 es : 609875.1584992459
Error de fold N°2 es : 609796.5623568108
Error de fold N°3 es : 607045.7600288235
Error de fold N°4 es : 606708.6256275818


In [23]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.2)

Error de fold N°0 es : 586627.6302466228
Error de fold N°1 es : 587471.2143684208
Error de fold N°2 es : 592331.3214685027
Error de fold N°3 es : 589452.9643709149
Error de fold N°4 es : 589999.2383762376


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(y_test, pred)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel("Valores reales", fontsize=16)
ax.set_ylabel("Valores predichos", fontsize=16);

In [ ]:
#reg = LinearRegression(normalize=True, n_jobs = -1)
#reg.fit(X,y)

In [ ]:
train.info()

In [ ]:
train['descripcion'] = train['descripcion'].to_string()
train.info()

In [ ]:
train.head()